## <font color='green'> Benchmarking - Graph Algorithms <font>

### <font color='green'> 1. Description<font>

We are using below datasets for Graph Algorithms.

 **preferentialAttachment:** 
 
  Dataset can be downloaded from https://sparse.tamu.edu/MM/DIMACS10/preferentialAttachment.tar.gz
  
  This graph has been generated following a preferential attachment process. Starting with a clique of five vertices, the vertices are successively added to the graph. Each new vertex chooses exactly five neighbors among the existing vertices, such that the probability of choosing a particular vertex is proportional to its degree.
 
 **caidaRouterLevel:**
 
  Dataset can be downloaded from https://sparse.tamu.edu/MM/DIMACS10/caidaRouterLevel.tar.gz
    
**coAuthorsDBLP:**
 
  Dataset can be downloaded from https://sparse.tamu.edu/MM/DIMACS10/coAuthorsDBLP.tar.gz
 
  This is a Co-Authorship Network where two authors are connected if they publish at least one paper together.

**dblp-2010:**
  
  Dataset can be downloaded from https://sparse.tamu.edu/MM/LAW/dblp-2010.tar.gz
  
  DBLP is a bibliography service from which an undirected scientific collaboration network can be extracted: each vertex  represents a scientist and two vertices are connected if they have worked together on an article. 

**citationCitesee:**

  Dataset can be downloaded from https://sparse.tamu.edu/MM/DIMACS10/citationCiteseer.tar.gz

  Citation network dataset published by DIMACS10. In this, the nodes represents a paper and edges represents a citation
    
**coPapersDBLP:**

  Dataset can be downloaded from https://sparse.tamu.edu/MM/DIMACS10/coPapersDBLP.tar.gz

  Citation network dataset published by DIMACS10. Here, the nodes represent scientists, edges represent collaborations (co-authoring a paper)
    
**coPapersCiteseer:**
 
  Dataset can be downloaded from https://sparse.tamu.edu/MM/DIMACS10/coPapersCiteseer.tar.gz

  Visualize coPapersCiteseer's link structure and discover valuable insights using our interactive graph visualization platform. Compare with hundreds of other networks across many different collections and types.

**as-Skitter:**
    
  Dataset can be downloaded from https://sparse.tamu.edu/MM/SNAP/as-Skitter.tar.gz
    
  This is Internet Topology datasets. Skitter is a tool for actively probing the Internet in order to analyze topology and performance. Skitter was also used in reference to the Macroscopic Topology Measurements Project and the Skitter infrastructure, which has since been replaced with the Archipelago (Ark) infrastructure.

### <font color='green'> 2. Data Preprocessing<font>

In [1]:
import os
import sys
import time
import numpy as np
import pandas as pd
import networkx as nx
import frovedis.graph as fnx
from collections import OrderedDict

In [2]:
# Please uncomment the below lines to download and unzip the dataset. We have removed the comments including the very first line with graph details

#dataset_links = {
#    'preferentialAttachment' : 'https://sparse.tamu.edu/MM/DIMACS10/preferentialAttachment.tar.gz',
#    'caidaRouterLevel'       : 'https://sparse.tamu.edu/MM/DIMACS10/caidaRouterLevel.tar.gz',
#    'coAuthorsDBLP'          : 'https://sparse.tamu.edu/MM/DIMACS10/coAuthorsDBLP.tar.gz',
#    'dblp-2010'              : 'https://sparse.tamu.edu/MM/LAW/dblp-2010.tar.gz',
#    'citationCiteseer'       : 'https://sparse.tamu.edu/MM/DIMACS10/citationCiteseer.tar.gz',
#    'coPapersDBLP'           : 'https://sparse.tamu.edu/MM/DIMACS10/coPapersDBLP.tar.gz',
#    'coPapersCiteseer'       : 'https://sparse.tamu.edu/MM/DIMACS10/coPapersCiteseer.tar.gz',
#    'as-Skitter'             : 'https://sparse.tamu.edu/MM/SNAP/as-Skitter.tar.gz'
#}
#for name, link in dataset_links.items():
#    !wget -N  {link}
#    !tar -xf {name + '.tar.gz'}
#    !mv {name}/{name+'.mtx'} datasets

# Load dataset
input_graph_files = {
    'preferentialAttachment' : './datasets/preferentialAttachment.mtx',
    'caidaRouterLevel'       : './datasets/caidaRouterLevel.mtx',
    'coAuthorsDBLP'          : './datasets/coAuthorsDBLP.mtx',
    'dblp'                   : './datasets/dblp-2010.mtx',
    'citationCiteseer'       : './datasets/citationCiteseer.mtx',
    'coPapersDBLP'           : './datasets/coPapersDBLP.mtx',
    'coPapersCiteseer'       : './datasets/coPapersCiteseer.mtx',
    'as-Skitter'             : './datasets/as-Skitter.mtx'
}

In [3]:
from frovedis.exrpc.server import FrovedisServer
FrovedisServer.initialize("mpirun -np 8 " + os.environ["FROVEDIS_SERVER"])

### <font color='green'> 3. Algorithm Evaluation<font>

#### 3.1 Graph Loading Demo

In [4]:
# -------- Graph loading demo --------
frov_data_loading_time = []
nx_data_loading_time = []
frov_graph = []
frov_graph_dir = []
nx_graph = []
nx_graph_dir = []
nnodes = []
nedges = []

for dataset, path in input_graph_files.items():
    dl_start_time = time.time()
    fgraph = fnx.read_edgelist(path, nodetype=np.int32, delimiter=' ')
    # for pagerank etc. we will use directed graph for better ranking
    fgraph_dir = fnx.read_edgelist(path, nodetype=np.int32, delimiter=' ', \
                                   create_using=nx.DiGraph())
    frov_data_loading_time.append(round(time.time() - dl_start_time, 4))
    frov_graph.append(fgraph)
    frov_graph_dir.append(fgraph_dir)
    
    dl_start_time = time.time()
    ngraph = nx.read_edgelist(path, nodetype=np.int32, delimiter=' ')
    # for pagerank etc. we will use directed graph for better ranking
    ngraph_dir = nx.read_edgelist(path, nodetype=np.int32, delimiter=' ', \
                                  create_using=nx.DiGraph())
    nx_data_loading_time.append(round(time.time() - dl_start_time, 4))
    nx_graph.append(ngraph)
    nx_graph_dir.append(ngraph_dir)

    nnodes.append(ngraph.number_of_nodes())
    nedges.append(ngraph.number_of_edges())

In [5]:
data_details = pd.DataFrame(OrderedDict({ "dataset": list(input_graph_files.keys()),
                                "num_nodes": nnodes,
                                "num_edges": nedges,
                                "frov_loading_time": frov_data_loading_time,
                                "nx_loading_time": nx_data_loading_time
                             }))
data_details["frov_speed_up"] = data_details["nx_loading_time"] / data_details["frov_loading_time"]
data_details

,dataset,num_nodes,num_edges,frov_loading_time,nx_loading_time,frov_speed_up
0,preferentialAttachment,100000,499985,0.4130,5.9338,14.367554
1,caidaRouterLevel,192244,609066,0.4993,7.5316,15.084318
2,coAuthorsDBLP,299067,977676,0.7140,12.3610,17.312325
3,dblp,300647,807700,0.7008,10.8511,15.483876
4,citationCiteseer,268495,1156647,0.6922,13.6544,19.726091
5,coPapersDBLP,540486,15245729,10.9483,142.8855,13.050930
6,coPapersCiteseer,434102,16036720,12.0299,152.6621,12.690222
7,as-Skitter,1696415,11095298,13.0069,152.0384,11.689057


#### 3.2 Single Source Shortest Path (SSSP)

In [6]:
src = 5
frov_sssp_time = []
nx_sssp_time = []
dist_matched = []

for i in range(len(frov_graph)):
    start_time = time.time()
    fpath, fdist = fnx.single_source_shortest_path(frov_graph[i], \
                   src, return_distance=True)
    frov_sssp_time.append(round(time.time() - start_time, 4))

    #print("frovedis: distance for {} dataset".format(dataset))
    #print(fdist)

    start_time = time.time()
    npath = nx.single_source_shortest_path(nx_graph[i], src)
    nx_sssp_time.append(round(time.time() - start_time, 4))

    ndist = {k: float(len(v)-1) for k, v in npath.items()}
    #print("networkx: distance for {} dataset".format(dataset))
    #print(ndist)

    dist_matched.append("Yes" if fdist == ndist else "No")

In [7]:
sssp_df = pd.DataFrame(OrderedDict ({ "dataset": list(input_graph_files.keys()),
                                      "frov_sssp_time": frov_sssp_time,
                                      "nx_sssp_time": nx_sssp_time,
                                      "result_matched": dist_matched
                                   }))
sssp_df["frov_speed_up"] = sssp_df["nx_sssp_time"] / sssp_df["frov_sssp_time"]
sssp_df

,dataset,frov_sssp_time,nx_sssp_time,result_matched,frov_speed_up
0,preferentialAttachment,0.0423,0.3782,Yes,8.940898
1,caidaRouterLevel,0.0887,0.6021,Yes,6.788050
2,coAuthorsDBLP,0.1059,24.6352,Yes,232.627007
3,dblp,0.0842,0.7880,Yes,9.358670
4,citationCiteseer,0.0948,1.1186,Yes,11.799578
5,coPapersDBLP,0.2040,4.3748,Yes,21.445098
6,coPapersCiteseer,0.1820,4.3377,Yes,23.833516
7,as-Skitter,0.7244,33.6921,Yes,46.510353


#### 3.3 PageRank

In [8]:
frov_pr_time = []
nx_pr_time = []
order_matched = []

#pagerank uses directed graph for generating better numerical ranks
for i in range(len(frov_graph_dir)):
    start_time = time.time()
    frov_ranks = fnx.pagerank(frov_graph_dir[i])
    frov_pr_time.append(round(time.time() - start_time, 4))
    df1 = pd.DataFrame({ "node": list(frov_ranks.keys()),
                         "rank": list(frov_ranks.values())
                       }) \
            .sort_values(by = ["rank", "node"]) \
            .head(10)
    #print(df1)

    start_time = time.time()
    nx_ranks = nx.pagerank(nx_graph_dir[i])
    nx_pr_time.append(round(time.time() - start_time, 4))
    df2 = pd.DataFrame({ "node": list(nx_ranks.keys()),
                         "rank": list(nx_ranks.values())
                       }) \
            .sort_values(by = ["rank", "node"]) \
            .head(10)
    #print(df2)
    order_matched.append("Yes" if list(df1.node.values) == list(df2.node.values) else "No")

In [9]:
pagerank_df = pd.DataFrame(OrderedDict ({ "dataset": list(input_graph_files.keys()),
                             "frov_pr_time": frov_pr_time,
                             "nx_pr_time": nx_pr_time,
                             "ranking_order_matched": order_matched
                          }))
pagerank_df["frov_speed_up"] = pagerank_df["nx_pr_time"] / pagerank_df["frov_pr_time"]
pagerank_df

,dataset,frov_pr_time,nx_pr_time,ranking_order_matched,frov_speed_up
0,preferentialAttachment,0.0226,10.7416,Yes,475.292035
1,caidaRouterLevel,0.0397,10.8338,Yes,272.891688
2,coAuthorsDBLP,0.0596,14.3700,Yes,241.107383
3,dblp,0.0838,11.0422,Yes,131.768496
4,citationCiteseer,0.0494,16.3914,Yes,331.809717
5,coPapersDBLP,0.0978,135.4959,Yes,1385.438650
6,coPapersCiteseer,0.1104,112.3445,Yes,1017.613225
7,as-Skitter,0.2974,112.0632,Yes,376.809684


#### 3.4 Breadth First Search (BFS)

In [10]:
source = 1
distance = 4 
frov_bfs_time = []
nx_bfs_time = []
result_matched = []

for i in range(len(frov_graph)):
    start_time = time.time()
    frov_res = fnx.descendants_at_distance(frov_graph[i], source, distance)
    frov_bfs_time.append(round(time.time() - start_time, 4))
    #print(list(frov_res)[:5])

    start_time = time.time()
    nx_res = nx.descendants_at_distance(nx_graph[i], source, distance)
    nx_bfs_time.append(round(time.time() - start_time, 4))
    #print(list(nx_res)[:5])

    result_matched.append("Yes" if len(frov_res - nx_res) == 0 else "No")

In [11]:
bfs_df = pd.DataFrame(OrderedDict ({ "dataset": list(input_graph_files.keys()),
                        "frov_bfs_time": frov_bfs_time,
                        "nx_bfs_time": nx_bfs_time,
                        "result_matched": result_matched
                     }))
bfs_df["frov_speed_up"] = bfs_df["nx_bfs_time"] / bfs_df["frov_bfs_time"]
bfs_df

,dataset,frov_bfs_time,nx_bfs_time,result_matched,frov_speed_up
0,preferentialAttachment,0.0169,0.3470,Yes,20.532544
1,caidaRouterLevel,0.0176,0.0864,Yes,4.909091
2,coAuthorsDBLP,0.0103,0.0025,Yes,0.242718
3,dblp,0.0091,0.0007,Yes,0.076923
4,citationCiteseer,0.0106,0.0087,Yes,0.820755
5,coPapersDBLP,0.0185,0.0467,Yes,2.524324
6,coPapersCiteseer,0.0144,0.0205,Yes,1.423611
7,as-Skitter,0.1782,3.8218,Yes,21.446689


#### 3.5 Connected Components (CC)

In [12]:
# -------- connected components demo --------
frov_cc_time = []
nx_cc_time = []
frov_ncomponents = []
nx_ncomponents = []
result_matched = []

for i in range(len(frov_graph)):
    start_time = time.time()
    tmp = fnx.connected_components(frov_graph[i])
    set_cc_frov = set()
    for elem in tmp:
        set_cc_frov.add(frozenset(elem))
    frov_cc_time.append(round(time.time() - start_time, 4))
    frov_ncomponents.append(len(set_cc_frov))

    start_time = time.time()
    tmp = nx.connected_components(nx_graph[i])
    set_cc_nx = set()
    for elem in tmp:
        set_cc_nx.add(frozenset(elem))
    nx_cc_time.append(round(time.time() - start_time, 4))
    nx_ncomponents.append(len(set_cc_nx))

    result_matched.append("Yes" if set_cc_frov == set_cc_nx else "No")

In [13]:
cc_df = pd.DataFrame(OrderedDict ({ "dataset": list(input_graph_files.keys()),
                        "frov_cc_time": frov_cc_time,
                        "nx_cc_time": nx_cc_time,
                        "frov_ncomponents": frov_ncomponents,
                        "nx_ncomponents": nx_ncomponents,
                        "result_matched": result_matched
                     }))
cc_df["frov_speed_up"] = cc_df["nx_cc_time"] / cc_df["frov_cc_time"]
cc_df

,dataset,frov_cc_time,nx_cc_time,frov_ncomponents,nx_ncomponents,result_matched,frov_speed_up
0,preferentialAttachment,0.0892,0.2807,1,1,Yes,3.146861
1,caidaRouterLevel,0.2706,0.5020,308,308,Yes,1.855137
2,coAuthorsDBLP,0.2603,0.7829,1,1,Yes,3.007683
3,dblp,3.1094,0.6905,22954,22954,Yes,0.222069
4,citationCiteseer,0.2537,0.9657,1,1,Yes,3.806464
5,coPapersDBLP,0.4972,3.4103,1,1,Yes,6.859010
6,coPapersCiteseer,0.4121,3.9138,1,1,Yes,9.497209
7,as-Skitter,1.6324,6.4308,756,756,Yes,3.939476


In [14]:
FrovedisServer.shut_down()